In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_groq import ChatGroq
models = {"Google": ["gemma2-9b-it", "gemma-7b-it"],
          "Groq": ["llama3-groq-70b-8192-tool-use-preview", "llama3-groq-8b-8192-tool-use-preview"],
          "Meta": ["llama-3.1-70b-versatile", "llama-3.1-8b-instant", "llama-3.2-1b-preview", "llama-3.2-3b-preview", "llama-3.2-11b-vision-preview", "llama-3.2-90b-vision-preview", "llama-guard-3-8b", "llama3-70b-8192", "llama3-8b-8192"],
          "Mistral": ["mixtral-8x7b-32768"],
          "OpenAI": ["whisper-large-v3", "whisper-large-v3-turbo"]}
llm = ChatGroq(model=models["Meta"][0], temperature=0)

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relavant documents from a vector 
database. By generating mulitple perspectives on the user quesion, your goal is to help 
the user overcome some of the limitations of the distance-based similarity search.
Provide these alternative questions seprated by newlines. Original question: {question}"""

multiquery_prompt = ChatPromptTemplate.from_template(template)
multiquery_prompt

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='You are an AI language model assistant. Your task is to generate five \ndifferent versions of the given user question to retrieve relavant documents from a vector \ndatabase. By generating mulitple perspectives on the user quesion, your goal is to help \nthe user overcome some of the limitations of the distance-based similarity search.\nProvide these alternative questions seprated by newlines. Original question: {question} '), additional_kwargs={})])

In [4]:
rag_template = """Answer the following question based on this context:
{context}
Question: {question}
"""
rag_prompt = ChatPromptTemplate.from_template(rag_template)
rag_prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the following question based on this context:\n{context}\nQuestion: {question}\n'), additional_kwargs={})])

In [5]:
# from langchain.document_loaders import UnstructuredPDFLoader
# pdf_path = r"D:\All Python\Pure-Python\P4\06-PromptEngineering\Project 3 - OpenAI\TP-ClassNote-6.pdf"
# loader = UnstructuredPDFLoader(file_path=pdf_path)
# pdf_docs = loader.load()
from langchain.document_loaders import WebBaseLoader
import bs4
loader = WebBaseLoader(web_path=("https://blog.langchain.dev/deconstructing-rag/"),
                       bs_kwargs=dict(
                           parse_only=bs4.SoupStrainer(
                               class_=("article-header section", "article-main section")
                           )
                       ))
blog_docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [33]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100,
                                               chunk_overlap=50,
                                               separators="\n")
blog_docs_splitted = text_splitter.split_documents(blog_docs)

In [34]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

vectorsote = Chroma.from_documents(documents=blog_docs_splitted,
                                   embedding=HuggingFaceEmbeddings())

retriever = vectorsote.as_retriever()
print(f"number of docs: {vectorsote._collection.count()}")
retriever.search_kwargs["k"] = vectorsote._collection.count()

number of docs: 15


In [35]:
from langchain_core.output_parsers import StrOutputParser

def generate_multi_queries():
    chain  = (
        multiquery_prompt
        | llm
        | StrOutputParser()
        | (lambda x: x.split("\n"))
    )
    return chain    

### Example of dumps and loads

In [36]:
from langchain.load import dumps, loads  # Using dumps from langchain.load

# Example input: a list of lists with dictionary documents
documents = [
    [{"name": "Alice", "age": 30}, {"name": "Bob", "age": 25}],
    [{"name": "Alice", "age": 30}],  # Duplicate document
    [{"name": "Charlie", "age": 35}]
]

# Function to serialize and flatten the documents
def get_unique_union1(documents: list[list]):
    # Serialize each document to a string format and flatten the list
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Use set to keep only unique serialized documents
    unique_docs = list(set(flattened_docs))
    return unique_docs

def get_unique_union2(documents: list[list]):
    flattened_docs = [loads(doc) for doc in documents]
    return flattened_docs

# Run the function
unique_documents = get_unique_union1(documents)
new = get_unique_union2(unique_documents)
print(unique_documents)
print(new)

['{"name": "Bob", "age": 25}', '{"name": "Alice", "age": 30}', '{"name": "Charlie", "age": 35}']
[{'name': 'Bob', 'age': 25}, {'name': 'Alice', 'age': 30}, {'name': 'Charlie', 'age': 35}]


In [37]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    
    unique_docs = list(set(flattened_docs))

    return [loads(doc) for doc in unique_docs]

In [ ]:
from langchain_core.runnables import RunnablePassthrough

def query(query, retriever):
    retrieval_chain = (generate_multi_queries() 
                       | retriever.map()
                       | get_unique_union)
    # docs = retrieval_chain.invoke({"question": quesion})
    # display(len(docs))
    # return docs
    final_rag_chain = ({"context": retrieval_chain, "question": RunnablePassthrough()} 
                       | rag_prompt
                       | llm
                       | StrOutputParser())
    return final_rag_chain.invoke(query)

In [51]:
from IPython.display import Markdown
answer = query("What is retrivial?", retriever)
display(Markdown(answer))

Retrieval is a core component of the new LLM (Large Language Model) operating system. It involves retrieving information from a data source (or sources) and loading it into the context window of the LLM, which is then used in LLM output generation, a process typically called retrieval augmented generation (RAG).